In [1]:
from model.my_models import SiameseUNetWithResnet50Encoder
from model.evaluate import evaluate
from utils.data_loading import SatelliteDataset
from utils.dice_score import dice_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses.focal import FocalLoss
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
from torchmetrics import ConfusionMatrix
from torch import optim

import math
import os
import numpy as np
import logging
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from pathlib import Path


In [2]:
# in tensorboard see the architecture of model 
# vit对输入图像大小有严格要求，需要是32的倍数，所以这里使用256*256的图像
from torchvision.models import resnet50
#tensorboard
#命令行输入 tensorboard --logdir=./training/runs
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./runs')
model = SiameseUNetWithResnet50Encoder()

# 定义输入张量，假设输入为两张 256x256 的 RGB 图像
input_1 = torch.rand(1, 3, 256, 256)
input_2 = torch.rand(1, 3, 256, 256)
# 不加torch.no_grad()会报错
with torch.no_grad():
    writer.add_graph(model, (input_1, input_2))
# with torch.no_grad():
#     writer.add_graph(model, input_to_model = torch.rand(1, 3, 256, 256))
writer.close()

C:\Users\liuyi\segment\damage-assessment\model\my_functions.py:34: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(diffY<0):


In [2]:
# img_home_path = "C:/Users/liuyi/segment/ubdd/xbd/Dataset"
img_home_path = "C:/Users/liuyi/segment/ubdd/xbd/Dataset_test"
post_dir_img = img_home_path + "/TierFull/Post/Image512/"
post_dir_mask = img_home_path + "/TierFull/Post/Label512/"
pre_dir_img = img_home_path + "/TierFull/Pre/Image512/"
pre_dir_mask = img_home_path + "/TierFull/Pre/Label512/"
post_dir_val_img = img_home_path + "/Validation/Post/Image512/"
post_dir_val_mask = img_home_path + "/Validation/Post/Label512/"
pre_dir_val_img = img_home_path + "/Validation/Pre/Image512/"
pre_dir_val_mask = img_home_path + "/Validation/Pre/Label512/"
# post_dir_img = Path('.\\Dataset\\TierFull\\Post\\Image512\\')
# post_dir_mask = Path('.\\Dataset\\TierFull\\Post\\Label512\\')
# pre_dir_img = Path('.\\Dataset\\TierFull\\Pre\\Image512\\')
# pre_dir_mask = Path('.\\Dataset\\TierFull\\Pre\\Label512\\')
# post_dir_val_img = Path('.\\Dataset\\Validation\\Post\\Image512\\')
# post_dir_val_mask = Path('.\\Dataset\\Validation\\Post\\Label512\\')
# pre_dir_val_img = Path('.\\Dataset\\Validation\\Pre\\Image512\\')
# pre_dir_val_mask = Path('.\\Dataset\\Validation\\Pre\\Label512\\')
dir_checkpoint = Path('./checkpoints/Vgg19SiamConc/')

In [3]:
closs = nn.CrossEntropyLoss()

floss = FocalLoss(mode = 'multiclass',
                alpha = None,
                gamma = 2.0,
                ignore_index = None,
                reduction = "mean",
                normalized = False,
                reduced_threshold = None)

In [31]:
#TODO:mask_suffix = '.png')是干啊的，在原文档的上一个版本的commit中能找到 ,increase = 8397
def get_class_weights():
    train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[1,1], True, True, True], probabilities = [.5,.5,.5,0,0,0,0])
    # 设置DataLoader
    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
    train_loader = DataLoader(train, shuffle=True, **loader_args)
     # 用于统计每个类别的频次，假设有5个类别
    classes = torch.zeros(5)
    with tqdm(total=len(train_loader), unit='img') as pbar:
        for batch in train_loader:
            image = batch['premask'].int() # 获取mask
            count = torch.bincount(torch.flatten(image),minlength = 5)
            classes = classes.add(count)
            pbar.update(image.shape[0])
    return torch.div(classes,torch.sum(classes))
print(get_class_weights())

  0%|          | 0/2800 [00:05<?, ?img/s]


KeyboardInterrupt: 

In [13]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torchmetrics
import numpy as np
import logging
from pathlib import Path
import torch.nn.functional as F

def train_net(net,
              device,
              start_epoch: int = 1,
              epochs: int = 5,
              batch_size: int = 1,
              learning_rate: float = 1e-5,
              train_percent: float = 0.5,
              val_percent: float = 0.1,
              save_checkpoint: bool = True,
              img_scale: float = 0.5,
              ampbool: bool = False,
              traintype: str = 'post',
              gradient_clipping: float = 1.0):
    # 1. Create dataset
    train = None
    validate = None

    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask, post_dir_img, post_dir_mask, 1, values=[[.8,1.5], True, True, True], probabilities=[0, 0, 0, 0])
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask, post_dir_val_img, post_dir_val_mask, 1, values=[[1, 1], False, False, False], probabilities=[0, 0, 0, 0])
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=batch_size, num_workers=1, pin_memory=True)
    
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))
    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

    # 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9, foreach=True)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,6,9,12,15,18,19,20,33,47,50,60,70,90,110,130,150,170,180,190], gamma=0.5)
    grad_scaler = torch.amp.GradScaler('cuda', enabled=ampbool)
    criterion = closs
    
    # 5. Initialize torchmetrics metrics
    train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=5, validate_args=False).to(device)
    train_precision = torchmetrics.Precision(task='multiclass', num_classes=5, average='macro', validate_args=False).to(device)
    train_recall = torchmetrics.Recall(task='multiclass', num_classes=5, average='macro', validate_args=False).to(device)
    
    

    # 6. Begin training
    for epoch in range(start_epoch, start_epoch + epochs):
        net.train()
        epoch_loss = 0
        epoch_steps = 0
        nancount = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                optimizer.zero_grad(set_to_none=True)
                preimage, postimage, post_masks, pre_masks = batch['preimage'], batch['postimage'], batch['postmask'], batch['premask']

                preimage = preimage.to(device=device, dtype=torch.float32)
                postimage = postimage.to(device=device, dtype=torch.float32)
                post_masks = post_masks.to(device=device, dtype=torch.long)
                pre_masks = pre_masks.to(device=device, dtype=torch.long)
                
                with torch.amp.autocast('cuda', enabled=ampbool):
                    masks_pred = None
                    if traintype == 'both':
                        masks_pred = net(preimage, postimage)
                        loss = criterion(masks_pred, post_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float()[:, 1:, ...],
                            F.one_hot(post_masks, 5).permute(0, 3, 1, 2).float()[:, 1:, ...],
                            multiclass=True
                        )
                    elif traintype == 'pre':
                        masks_pred = net(preimage)
                        loss = criterion(masks_pred, pre_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float(),
                            F.one_hot(pre_masks, 2).permute(0, 3, 1, 2).float(),
                            multiclass=True
                        )
                    elif traintype == 'post':
                        masks_pred = net(postimage)
                        loss = criterion(masks_pred, post_masks)
                        loss += dice_loss(
                            F.softmax(masks_pred, dim=1).float()[:, 1:, ...],
                            F.one_hot(post_masks, 5).permute(0, 3, 1, 2).float()[:, 1:, ...],
                            multiclass=True
                        )

                grad_scaler.scale(loss).backward()
                grad_scaler.step(optimizer)
                grad_scaler.update()
                
                # Update metrics
                train_accuracy.update(masks_pred, post_masks)
                train_precision.update(masks_pred, post_masks)
                train_recall.update(masks_pred, post_masks)
                
                pbar.update(postimage.shape[0])
                epoch_steps += 1 #已经处理的批次数
                
                if math.isnan(loss.item()):
                    epoch_loss+=0
                    nancount += 1
                else:
                    epoch_loss += loss.item()   
                    
                # if epoch_steps == 1 or epoch_steps % max(1, len(train_loader) // 10) == 0:
                pbar.set_postfix(**{
                    'loss (batch)': float(loss.item()),
                    'loss': float(epoch_loss / epoch_steps),
                    'accuracy': float(train_accuracy.compute().item()),
                    'precision': float(train_precision.compute().item()),
                    'recall': float(train_recall.compute().item())
                })
                
        # 计算最终的训练指标
        train_acc = train_accuracy.compute()
        train_prec = train_precision.compute()
        train_rec = train_recall.compute()
        train_loss = epoch_loss / len(train_loader)
        # 使用现有的evaluate函数进行验证
        val_score, val_class_scores, val_loss = evaluate(net, dataloader=val_loader, device=device, ampbool=ampbool, traintype=traintype)
        scheduler.step()
        
        print(f'Epoch {epoch}')
        print(f'Training - Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}, Recall: {train_rec:.4f}')
        print(f'Training Loss: {train_loss:.4f}')
        print(f'Validation - Dice Score: {val_score:.4f}')
        print(f'Validation - Class Dice Scores: {[f"{score:.4f}" for score in val_class_scores]}')
        print(f'Validation Loss: {val_loss:.4f}')
        print(f'NaN Count: {nancount}')

        if save_checkpoint:
            Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
            torch.save(net.state_dict(), str(dir_checkpoint / f'branch12checkpoint_epoch_{epoch}.pth'))
            logging.info(f'Checkpoint {epoch} saved!')


In [14]:
classes = 5
bilinear = True
loadstate = False
# loadstate = True
load = './checkpoints/Vgg19SiamConc/checkpoint_epoch12.pth'
start_epoch = 0
# start_epoch = 13
epochs = 20
batch_size = 1
lr = 1e-6
scale = 1
train =0.15259598603*2
val = 1
ampbool = True
save_checkpoint = True
traintype = 'both'
gradclip = 1.0

In [15]:
def confusionmatrix(pred,true):
    result = np.zeros((5,5))
    for i in range(true.shape[1]):
        for j in range(true.shape[2]):
            result[true[0][i][j]][pred[0][i][j]] +=1
    return result

In [16]:
def modelconfusionmatrix(filepath,net,train_percent,val_percent):
    device = 'cuda'
    net.load_state_dict(torch.load(filepath, map_location=device))
    net.to(device=device)
    net.eval() # 评估模式
    
    # 新增的
    train = None
    validate = None
    
    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0])
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0])
        # train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0],increase = 0 ,mask_suffix = '.png',normalizemodel = 'vgg19')
        # validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0],increase = 0,mask_suffix = '.png', normalizemodel = 'vgg19')
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
        
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)    

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))    

    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args) 
    resulttrain = torch.zeros(5,5).to(device)
    resultval = torch.zeros(5,5).to(device)

    num_val_batches = len(val_loader)
    num_train_batches = len(train_loader)
    confmat = ConfusionMatrix(task="multiclass", num_classes=5).to(device)

    with tqdm(total=num_train_batches, desc='train', unit='img') as pbar:
        for batch in train_loader:
            preimage, postimage, true_masks = batch['preimage'], batch['postimage'], batch['postmask']
            # preimage, postimage, true_masks = batch['preimage'], batch['image'], batch['mask']
            preimage = preimage.to(device=device, dtype=torch.float32)
            postimage = postimage.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.long)
            mask_true = F.one_hot(true_masks, 5).permute(0,3, 1, 2).float()
            with torch.amp.autocast('cuda', enabled = True):
                # predict the mask
                mask_pred = net(preimage,postimage)
                # convert to one-hot format
                pred = F.softmax(mask_pred, dim=1).int().argmax(-3)
                true = F.one_hot(true_masks, 5).int().permute(0, 3, 1, 2).argmax(-3)
                resulttrain += confmat(pred,true)
            pbar.update(postimage.shape[0])
    print(resulttrain)
    with tqdm(total=num_val_batches, desc='validation', unit='img') as pbar:
        for batch in val_loader:
            preimage, postimage, true_masks = batch['preimage'], batch['postimage'], batch['postmask']
            preimage = preimage.to(device=device, dtype=torch.float32)
            postimage = postimage.to(device=device, dtype=torch.float32)
            true_masks = true_masks.to(device=device, dtype=torch.long)
            mask_true = F.one_hot(true_masks, 5).permute(0,3, 1, 2).float()
            with torch.cuda.amp.autocast(enabled = True):
                # predict the mask
                mask_pred = net(preimage,postimage)
                # convert to one-hot format
                pred = F.softmax(mask_pred, dim=1).int().argmax(-3)
                true = F.one_hot(true_masks, 5).int().permute(0, 3, 1, 2).argmax(-3)
                resultval += confmat(pred,true)
            pbar.update(postimage.shape[0])
    print(resultval)
    net.train()

    # Fixes a potential division by zero error
    if num_val_batches == 0:
        return resulttrain,resultval
    return resulttrain,resultval

In [17]:
def EvaluateFolder(folderpath,net,train_percent,val_percent):
    in_files = [folderpath + s for s in os.listdir(folderpath)]
    try:
        train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0])
        # train = SatelliteDataset(pre_dir_img, pre_dir_mask,post_dir_img, post_dir_mask, 1, values =  [[.8,1.5], True, True,True], probabilities = [0,0,0,0],increase = 0 ,mask_suffix = '.png',normalizemodel = 'vgg19')
        validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0]) 
        # validate = SatelliteDataset(pre_dir_val_img, pre_dir_val_mask,post_dir_val_img, post_dir_val_mask, 1, values =  [[1,1], False, False, False], probabilities = [0,0,0,0],increase = 0,mask_suffix = '.png', normalizemodel = 'vgg19')
    except (AssertionError, RuntimeError):
        print('error')

    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
        
    # 2. Split into train / validation partitions
    n_train = int(len(train) * train_percent)
    n_train_none = int(len(train) - n_train)
    n_val = int(len(validate) * val_percent)
    n_val_none = int(len(validate) - n_val)

    train_set, train_val_none_set = random_split(train, [n_train, n_train_none], generator=torch.Generator().manual_seed(0))
    
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)    

    val_set, val_none_set = random_split(validate, [n_val, n_val_none], generator=torch.Generator().manual_seed(0))    

    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)
    
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    
    loader_args = dict(batch_size=1, num_workers=1, pin_memory=True)        
    
    for i in in_files:
        net.load_state_dict(torch.load(i, map_location=device))
        net.to(device=device)
        val_score, val_classes,val_loss = evaluate(net,dataloader = val_loader,device = device,ampbool = True,traintype = 'both')
        train_score, train_classes,train_loss = evaluate(net,dataloader = train_loader,device = device,ampbool = True,traintype = 'both')
        print('Train: ' + i)
        print(train_score)
        print(train_classes)
        print(train_loss)
        print('Validation: ' + i)
        print(val_score)
        print(val_classes)
        print(val_loss)

In [18]:
# net = SiamUNetConCVgg19()
net = SiameseUNetWithResnet50Encoder()

In [19]:
EvaluateFolder('./checkpoints/Final/',net,train,val)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './checkpoints/Final/'

In [20]:
model_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [21]:
mem_params = sum([param.nelement()*param.element_size() for param in net.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in net.buffers()])
mem = mem_params + mem_bufs
mem

553700636

In [22]:

if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    #net = SiamUNetDiff()
    #net = SiamUNetConC()
    #net = SiameseUNetWithResnet50Encoder()
    #net = SiamUnet_diff_Full(3,5)
    #net = UNetWithResnet50Encoder(5,'custom','./checkpoints/Resnet50ContrastiveWeight/checkpoint_epoch0.pth')
    #net = UNet(out_classes=classes, up_sample_mode='conv_transpose',batch_norm=False)
    #net = SiameseUNet(n_channels=3, n_classes = classes, bilinear=bilinear)
    #net = UNetVgg19V2(out_classes=classes, up_sample_mode='conv_transpose')
    #net = vgg19bn_unet(5,True)
    #net = vgg19nobn_unet(5,True)
    #net = vgg19nobn_unetdouble(5,True)
    #net = VGGUnet19nobnspace(out_channels = 5, pretrained = True)
    #net = SiamUNetCombVgg19()
    ### 之前是用这个 net = SiamUNetConCVgg19()
    #net = resnet_unet()
    #net = VGG16Unet(out_channels = 5)
    #net = resnet_siamunet()
    #net = SiamUNetDiffVgg19()
    #net = SiamUNetFullConCVgg19()
    #net = SiamUNetConCResnet50(True,5)
    #net = UNETResnet50(True,2)
    '''net = smp.Unet(
        encoder_name='resnext101_32x8d',
        encoder_depth=5,
        encoder_weights='imagenet',
        decoder_use_batchnorm=False,
        decoder_channels=(1024,512,256,128, 64),
        decoder_attention_type=None,
        in_channels=3,
        classes=5,
        activation=None,
        aux_params=None
    )
    '''
    if loadstate:
        net.load_state_dict(torch.load(load, map_location=device))
        logging.info(f'Model loaded from {load}')

    net.to(device=device)
    train_net(net=net,
                  start_epoch = start_epoch,
                  epochs=epochs,
                  batch_size=batch_size,
                  learning_rate=lr,
                  device=device,
                  img_scale=scale,
                  train_percent = train,
                  val_percent=val,
                  save_checkpoint = save_checkpoint,
                  ampbool = ampbool,
                  traintype = traintype,
                  gradient_clipping=gradclip)

validation:  33%|███▎      | 1/3 [00:29<00:58, 29.31s/img, loss (batch)=2.55]


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "C:\Users\liuyi\AppData\Local\anaconda3\envs\xbdtwostep\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 38, in do_one_step
    data = pin_memory(data, device)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\liuyi\AppData\Local\anaconda3\envs\xbdtwostep\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 69, in pin_memory
    clone.update({k: pin_memory(sample, device) for k, sample in data.items()})
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\liuyi\AppData\Local\anaconda3\envs\xbdtwostep\Lib\site-packages\torch\utils\data\_utils\pin_memory.py", line 69, in <dictcomp>
    clone.update({k: pin_memory(sample, device) for k, sample in data.items()})
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\liuyi\AppData\Local\anaconda3\envs\xbdtwostep\Lib\site-packages\torch\utils\da